In [ ]:
# Activar el entorno de conda
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential

# Modelo

In [3]:
# Tamaño al que se redimensionarán las imágenes y tamaño de batch
img_size = (128, 128)
batch = 32

# Ajuste automático del pipeline tf.data para solapar CPU con GPU
AUTOTUNE = tf.data.AUTOTUNE

def make_ds(path, shuffle=False):
    """
    Carga un dataset desde una estructura de carpetas:
      path/
        clase_1/
        clase_2/
        ...
    Redimensiona a img_size, agrupa en batches y aplica cache+prefetch para eficiencia.
    'shuffle=True' solo para el split de entrenamiento.
    """
    ds = keras.utils.image_dataset_from_directory(
        path,
        image_size=img_size,   # redimensiona cada imagen a (180,180)
        batch_size=batch,      # tamaño de lote
        shuffle=shuffle        # barajar solo en train; en val/test mantener orden estable
    )
    # cache(): guarda en RAM (o disco si se pasa una ruta) para evitar decodificar cada epoch
    # prefetch(): solapa el preprocesamiento con el entrenamiento
    return ds.cache().prefetch(AUTOTUNE)

# Crea los tres splits a partir de carpetas separadas
train_ds = make_ds("data/train", shuffle=True)
val_ds   = make_ds("data/val",   shuffle=False)
test_ds  = make_ds("data/test",  shuffle=False)

# Comprueba que el orden y los nombres de clases sean consistentes entre splits
class_names = train_ds.class_names
assert class_names == val_ds.class_names == test_ds.class_names, "Clases no coinciden entre splits"

model = Sequential([
    # Normaliza píxeles uint8 [0..255] -> float32 [0..1]
    layers.Rescaling(1./255, input_shape=img_size + (3,)),
    
    # Bloque conv 1
    layers.Conv2D(32, 3, padding="same", kernel_initializer="he_uniform"),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.1),
    layers.MaxPooling2D(),

    # Bloque conv 2
    layers.Conv2D(64, 3, padding="same", kernel_initializer="he_uniform"),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.1),
    layers.MaxPooling2D(),

    # Bloque conv 3
    layers.Conv2D(128, 3, padding="same", kernel_initializer="he_uniform"),
    layers.BatchNormalization(),
    layers.LeakyReLU(alpha=0.1),
    layers.MaxPooling2D(),

    # GAP reduce parámetros
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),

    # Head denso
    layers.Dense(256, kernel_initializer="he_uniform"),
    layers.LeakyReLU(alpha=0.1),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(128, kernel_initializer="he_uniform"),
    layers.LeakyReLU(alpha=0.1),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(3, activation="softmax"),
])

# lr(t) = 0.001 * (0.9)^(step/1000)
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=False
)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

# Compila el modelo con pérdida "sparse" (etiquetas enteras 0..2) y accuracy como métrica
model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

NotFoundError: Could not find directory data/train

# Entrenamiento

In [ ]:
# Entrena el modelo registrando métricas en history para análisis posterior
history = model.fit(train_ds, validation_data=val_ds, epochs=20)

# Evaluación objetiva en el conjunto de test

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test — loss: {test_loss:.4f}  acc: {test_acc:.4f}")

# Métricas

In [ ]:
# y_true: concatena las etiquetas reales de cada batch del test
y_true = np.concatenate([y.numpy() for _, y in test_ds], axis=0)

# y_pred: para cada imagen, argmax de las probabilidades softmax -> clase predicha
y_pred = np.argmax(model.predict(test_ds), axis=1)

# Genera la matriz de confusión (filas: clase real, columnas: predicción)
cm = tf.math.confusion_matrix(y_true, y_pred, num_classes=len(class_names))
print("Confusion matrix:\n", cm.numpy())